# StyleTTS2 Training on Google Colab
## Train Cigdem TTS Model

This notebook will help you train your Turkish TTS model on Google Colab with free GPU.

## Step 1: Check GPU and Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi
import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Step 2: Clone StyleTTS2 Repository

In [ ]:
# Clone your repository
!git clone https://github.com/ElkhanAbbasov/TTS_Cigdem.git
%cd TTS_Cigdem

## Step 3: Install Dependencies

In [ ]:
# Install required packages
!pip install -q SoundFile torchaudio munch pydub pyyaml librosa nltk matplotlib accelerate transformers einops tqdm
!pip install -q git+https://github.com/resemble-ai/monotonic_align.git

print("✅ All dependencies installed!")

## Step 4: Mount Google Drive (Optional - for saving checkpoints)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create checkpoint backup folder in Drive
!mkdir -p '/content/drive/MyDrive/Cigdem_TTS_Checkpoints'
print("✅ Google Drive mounted!")

## Step 5: Verify Training Data

In [ ]:
# Check if training data exists
import os

train_list = "Data/my_train_list.txt"
if os.path.exists(train_list):
    with open(train_list, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    print(f"✅ Training data found: {len(lines)} samples")
    print("\nFirst 3 samples:")
    for line in lines[:3]:
        print(f"  {line.strip()}")
else:
    print("❌ Training data not found! Please upload your data.")

## Step 6: Check Configuration

In [ ]:
# Display current config
import yaml

with open('Configs/config_ft.yml', 'r') as f:
    config = yaml.safe_load(f)

print("📋 Current Configuration:")
print(f"  Epochs: {config['epochs']}")
print(f"  Batch size: {config['batch_size']}")
print(f"  Save frequency: Every {config['save_freq']} epochs")
print(f"  Pretrained model: {config.get('pretrained_model', 'None')}")

## Step 7: Start Training

**IMPORTANT:** 
- Free Colab sessions last ~12 hours max
- Save checkpoints frequently to Google Drive
- You can resume training later from saved checkpoints

In [ ]:
# Start training
!python train_finetune.py --config_path Configs/config_ft.yml

## Step 8: Backup Checkpoints to Google Drive (Run periodically)

In [ ]:
# Copy checkpoints to Google Drive
!cp -r Models/Cigdem_TTS/epoch_*.pth '/content/drive/MyDrive/Cigdem_TTS_Checkpoints/' 2>/dev/null || echo "No checkpoints to backup yet"
!cp Models/Cigdem_TTS/train.log '/content/drive/MyDrive/Cigdem_TTS_Checkpoints/' 2>/dev/null || echo "No log file yet"

# List backed up files
!ls -lh '/content/drive/MyDrive/Cigdem_TTS_Checkpoints/'
print("\n✅ Checkpoints backed up to Google Drive!")

## Step 9: View Training Progress

In [ ]:
# Display last 20 lines of training log
!tail -n 20 Models/Cigdem_TTS/train.log

## Step 10: Test Model (After training reaches epoch 20+)

In [ ]:
# Test the model
# First, update test_cigdem.py to use the latest checkpoint

import glob
checkpoints = sorted(glob.glob('Models/Cigdem_TTS/epoch_*.pth'))
if checkpoints:
    latest = checkpoints[-1]
    print(f"Latest checkpoint: {latest}")
    
    # You can modify and run test_cigdem.py here
    # For now, just show available checkpoints
    print("\nAvailable checkpoints:")
    for cp in checkpoints:
        print(f"  {cp}")
else:
    print("No checkpoints found yet. Continue training!")

## Step 11: Download Results

In [ ]:
# Zip checkpoints for download
!zip -r cigdem_tts_checkpoints.zip Models/Cigdem_TTS/epoch_*.pth Models/Cigdem_TTS/train.log

from google.colab import files
files.download('cigdem_tts_checkpoints.zip')

print("✅ Checkpoints ready for download!")

## Tips for Using Colab:

1. **Runtime limits**: Free Colab disconnects after ~12 hours or 90 mins of inactivity
2. **Save frequently**: Run the backup cell (Step 8) every 2-3 hours
3. **Resume training**: If disconnected, re-run all cells and training will resume from last checkpoint
4. **GPU quota**: Free Colab has limited GPU hours per week (~15-20 hours)
5. **Colab Pro**: Consider upgrading for longer sessions and better GPUs

## How to Resume After Disconnect:

1. Restore checkpoints from Google Drive:
```python
!cp '/content/drive/MyDrive/Cigdem_TTS_Checkpoints/epoch_*.pth' Models/Cigdem_TTS/
```

2. Update config to point to latest checkpoint

3. Re-run training cell